<a href="https://colab.research.google.com/github/lucas1619/tf-complejidad/blob/master/TP_Documentacion_Codigo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Documentación del proyecto "Quoridor"

### Desarrollo de la clase DoubleLinkedList
Esta **clase** contiene la implementación de una **lista doblemente enlazada** con sus diferentes métodos, tales como insertar al principio, insertar al final, buscar por valor, eliminar al prinicipio y eliminar al final, esta clase nos servirá para implementar nuestra clase Tablero, la cual es el grafo.

In [ ]:
import gc
class Node:
    def __init__(self, value=None):
        self.value = value
        self.next = None
        self.prev = None
class DoubleLinkedList:

    def __init__(self):
        self.start = Node()
        self.end = Node()
        self.size = 0
    def __contains__(self, value):
        aux = self.start
        while aux != None:
            if aux.value == value:
                return True
        aux = aux.next
        return False
    def __len__(self):
        return self.size

    def __iter__(self):
        node = self.start
        while node:
            yield node.value
            node = node.next

    def push_front(self, value):
        new = Node(value)
        if self.size == 0:
            self.start = self.end = new
            self.size += 1
            return
        new.next = self.start
        self.start.prev = new
        self.start = new
        self.size += 1

    def push_back(self, value):
        if self.size == 0:
            self.push_front(value)
            return
        new = Node(value)
        self.end.next = new
        new.prev = self.end
        self.end = new
        self.size += 1

    def insert(self, index, value):
        if self.size == 0 or index == 0:
            self.push_front(value)
            return
        if index == self.size - 1:
            self.push_back(value)
            return
        if index < 0 or index >= self.size:
            return
        new = Node(value)
        c = self.start
        for i in range(index - 1):
            c = c.next
        new.next = c.next
        c.next.prev = new
        c.next = new
        new.prev = c
        self.size += 1

    def pop_back(self):
        if self.size <= 0:
            return
        if self.size == 1:
            self.start = self.end = None
            self.size -= 1
            gc.collect()  # eliminar memoria sin referencia
            return
        self.end = self.end.prev
        self.end.next.prev = None
        self.end.next = None
        self.size -= 1
        gc.collect()  # eliminar memoria sin referencia

    def pop_front(self):
        if self.size <= 1:
            self.pop_back()
            return
        self.start = self.start.next
        self.start.prev.next = None
        self.start.prev = None
        self.size -= 1
        gc.collect()

    def delete(self, index):
        # lo hacen uds, eliminar en una posicion :v
        self.size -= 1

    def erase(self, pos):
        if pos < self.size and pos >= 0:
            if (pos == 0):
                self.pop_front()
            elif pos == self.size:
                self.pop_back()
            else:
                punt = self.start
                contador = 0
                while (contador != pos - 1):
                    punt = punt.next
                    contador += 1
                    elimi = punt.next
                punt.next = elimi.next
                elimi.next = None
                del elimi
                self.size -= 1

    def buscar_por_valor(self, valor):
        contador = 0
        punt = self.start
        while (contador != self.size):
            if punt.value == valor:
                return contador
            punt = punt.next
            contador += 1
        return -1


###Desarrollo de la clase Tablero

Esta clase contiene el grafo para el juego, el grafo se compone de un vector de listas doblemente enlazadas, lo formamos así con el objetivo de tener un acceso $O(1)$ a los elementos y un insertado dinámico.

La función $__init__$ inicializa el grafo, es aquí donde se hace cada conexión, la cual se representará en forma de una lista de adyacencia.



In [ ]:
class Tablero:
    def __init__(self, n):
        self.tam = 792/n
        self.n = n
        self.q_nodos = n * n
        self.grafo = [DoubleLinkedList() for i in range(self.q_nodos)]
        for i, nodo in enumerate(self.grafo):
            if i % n != n - 1:
                self.grafo[i].push_back(i + 1)
                self.grafo[i + 1].push_back(i)
            if i + n < n * n:
                self.grafo[i].push_back(i + n)
                self.grafo[i + n].push_back(i)
    

La función $_get_coord_$ que recibe como parametro un valor obtiene ....

In [ ]:
    def get_coord(self, value):
          coord = {"x": None, "y": None}
          coord['y'] =  value//self.n
          coord['x'] = value %self.n
          return coord 

La función $graficar_tablero_$ que recibe como parametros a la pantalla y a la librería pygame ....

In [ ]:
    def graficar_tablero(self, pantalla, pygame):
          coord = None
          for i, _ in enumerate(self.grafo):
              coord = self.get_coord(i)
              pygame.draw.rect(pantalla, negro, (coord['x']*self.tam, coord['y']*self.tam, self.tam, self.tam), 1)


Por último, la funcion $conectados_$ que recibe dos nodos como atributo determina si un nodo está conectado con el otro, debido a que el grafo tiene forma de matriz, si un nodo1 está conectado con un nodo2, el nodo2 también estará conectado con el nodo 1.

In [ ]:
    def conectados(self,nodo1,nodo2):
          return nodo2 in self.grafo[nodo1]

###Desarrollo de la clase Quoridor
Este contiene el juego donde se inicializa con la cantidad de jugadores y renderiza la parte gráfica 

La función $__init__$ recibe como parametro el grafo, el lado del grafo n, la cantidad de jugadores. El cual se encarga de inicializar el juego Quoridor con la cantidad de jugadores y la posición en la que iniciaran. Ademas de determinar el turno de cada jugador

In [ ]:
    def __init__(self, n, qjugadores):
        self.turno = 0
        self.tablero = Tablero(n)
        self.ganador = False
        self.lista_de_jugadores = []
        self.lista_de_jugadores.append(
            Jugador(1, self.tablero.grafo, self.tablero.n // 2, (self.tablero.q_nodos - self.tablero.n // 2)))
        self.lista_de_jugadores.append(
            Jugador(1, self.tablero.grafo, (self.tablero.q_nodos - self.tablero.n // 2) , self.tablero.n // 2))
        if qjugadores == 4:
            self.lista_de_jugadores.append(
                Jugador(1, self.tablero.grafo, (self.tablero.n * (self.tablero.n / 2 + 1)) - 1,
                        (self.tablero.n * (self.tablero.n / 2 + 1)) - self.tablero.n))
            self.lista_de_jugadores.append(
                Jugador(3, self.tablero.grafo, (self.tablero.n * (self.tablero.n / 2 + 1)) - self.tablero.n,
                        (self.tablero.n * (self.tablero.n / 2 + 1)) - 1))

La función $_start_$ recibe como parametro el grafo se encarga de renderizar el tablero con los jugadores gracias a la libreria Pygames


In [ ]:
def start(self):
        pygame.init()
        pygame.display.set_caption(u'Quoridor')
        pantalla = pygame.display.set_mode((792, 792))
        clock = pygame.time.Clock()
        is_running = True
        pantalla.fill(blanco)
        self.tablero.graficar_tablero(pantalla, pygame)
        for i, jugador in enumerate(self.lista_de_jugadores):
            jugador.graficar(pantalla, pygame, self.tablero.n, self.tablero.tam,colores[i])
        pygame.display.update()
        pygame.time.wait(3000)  
        while is_running:
            
            if(self.ganador == True):
                break
            clock.tick(1)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    is_running = False  
            pantalla.fill(blanco)
            self.tablero.graficar_tablero(pantalla, pygame)
            for i, jugador in enumerate(self.lista_de_jugadores):
                jugador.graficar(pantalla, pygame, self.tablero.n, self.tablero.tam,colores[i])
            self.lista_de_jugadores[self.turno].piensa(self.lista_de_jugadores[(self.turno + 1) % 2], self.tablero)
            self.ganador = self.lista_de_jugadores[self.turno].mueve(self.lista_de_jugadores[(self.turno + 1) % 2], self.tablero)
            self.turno = (self.turno + 1) % 2
            pygame.display.update()

        pantalla.fill(blanco)
        self.tablero.graficar_tablero(pantalla, pygame)
        for i, jugador in enumerate(self.lista_de_jugadores):
            jugador.graficar(pantalla, pygame, self.tablero.n, self.tablero.tam,colores[i])
        pygame.display.update()        
        pygame.time.wait(3000)                              
        pygame.quit()        

###Desarrollo de la clase Pensamiento

Se encarga de m



In [ ]:
def __init__(self, Grafo, indica):
        self.grafo = Grafo
        self.indica = indica

Se encarga de actualizar la ruta $_actualiza_ruta_$ 

In [ ]:
def actualiza_ruta(self, start, end):
        if self.indica == 1:
            return self.bellman_ford(start, end)
        elif self.indica == 2:
            return self.dijkstra(start, end)
        else:
            return self.astar(start, end)

Algoritmo dijstra es uno de los algortimos de búsqueda de camino mas corto que usamos $_dijstra_$  tiene como parametros el grafo y los nodos que representan el start y el end 

In [ ]:
def dijsktra(self, Nodos):

     S = []
     Queue = []
     anterior = [0 for i in range(max(grafo) +1)]
     distancia = [0 for i in range(max(grafo) +1)]

     for nodo in grafo:
         distancia[nodo] = 10000
         Queue.append(nodo)
     distancia[Nodos[0]] = 0

     while not len(Queue) == 0:
        distancia_minima = 10000
        for nodo in Queue:
            if distancia[nodo] < distancia_minima:
                distancia_minima = distancia[nodo]
                nodo_temporal = nodo
        nodo_distancia_minima = nodo_temporal
        Queue.remove(nodo_distancia_minima)

        for vecino in grafo[nodo_distancia_minima]:
            if distancia[nodo_distancia_minima] == 10000:
                distancia_temporal = 0
            else:
                distancia_temporal = distancia[nodo_distancia_minima]
            distancia_con_peso = distancia_temporal + 1
            if distancia_con_peso < distancia[vecino]:
                distancia[vecino] = distancia_con_peso
                anterior[vecino] = nodo_distancia_minima

        if nodo_distancia_minima == Nodos[1]:
            if anterior[nodo_distancia_minima] != 0 or nodo_distancia_minima == Nodos[0]:
                while nodo_distancia_minima != 0:
                    S.insert(0, nodo_distancia_minima)
                    nodo_distancia_minima = anterior[nodo_distancia_minima]
                return S          

Algoritmo bellman_ford es uno de los algortimos de búsqueda de camino mas corto que usamos $_bellman_ford_$  tiene como parametros el grafo y el inicio y fin de los nodos. 

In [ ]:
def bellman_ford(self, start, end):
        ruta = DoubleLinkedList()
        ruta.push_front(end)

        distancia = [float("Inf") for x in range(len(self.grafo))]

        previo = [-1 for x in range(len(self.grafo))]
        distancia[start] = 0
        for iteracion in range(len(self.grafo) - 1):
            for actual, nodo in enumerate(self.grafo):
                for adyacente in nodo:
                    if distancia[actual] + 1 < distancia[adyacente]:
                        distancia[adyacente] = distancia[actual] + 1
                        previo[adyacente] = actual
        anterior = previo[end]
        while anterior != start:
            ruta.push_front(anterior)
            anterior = previo[anterior]
        return list(ruta)

Algoritmo astar es uno de los algortimos de búsqueda de camino mas corto que usamos $_astar_$  tiene como parametros el grafo y el inicio y fin de los nodos. 

In [ ]:
def astar(self, start, goal):  # self.n

        def heuristic(current, goal):
            h = abs(current[0] - goal[0]) + abs(current[1] - goal[1])
            return float(h)

        def get_heuristic_value(node):
            return node.globhal

        current = start
        c = start % self.grafo.n + start // self.grafo.n
        f = start // self.grafo.n
        c1 = goal % self.grafo.n + goal // self.grafo.n
        f1 = goal // self.grafo.n
        grafoo = [Node_Astar(i) for i in range(self.grafo.n * self.grafo.n)]
        grafoo[start].local = 0.0
        grafoo[start].globhal = heuristic((c, f), (c1, f1))
        not_tested_nodes = [grafoo[start]]
        ruta = []

        while len(not_tested_nodes) != 0 and current != goal:

            not_tested_nodes.sort(key=get_heuristic_value)

            while len(not_tested_nodes) != 0 and not_tested_nodes[0].visited:
                not_tested_nodes.pop(0)

            if len(not_tested_nodes) == 0:
                break

            current = not_tested_nodes[0].pos
            ruta.append(grafoo[current].padre)
            not_tested_nodes[0].visited = True

            for neighbor in self.grafo[current]:

                if not grafoo[neighbor].visited:
                    not_tested_nodes.append(grafoo[neighbor])

                sum = float(grafoo[current].local + 1)

                if sum < grafoo[neighbor].local:
                    grafoo[neighbor].padre = current
                    grafoo[neighbor].local = sum
                    c = neighbor % self.grafo.n + neighbor // self.grafo.n
                    f = neighbor // self.grafo.n
                    grafoo[neighbor].globhal = grafoo[neighbor].local + \
                        heuristic((c, f), (c1, f1))

        ruta.append(goal)
        ruta.pop(0)
        return ruta

### Desarrollo de la clase Jugador


Función $_init_$ recibe como parámetros al grafo, la variable indicar, g 

In [ ]:
  def __init__(self, indica, Grafo, nodostart, nodogoal):
        self.nodogoal = nodogoal
        self.pensamiento = Pensamiento(Grafo, indica)
        self.lista_ruta = []
        self.lista_ruta_rival = []
        self.current = nodostart
        self.primero = True

In [ ]:
def piensa(self, rival, tablero):
        if self.primero:
            self.lista_ruta = self.pensamiento.actualiza_ruta(self.current, self.nodogoal)
            self.lista_ruta_rival = self.pensamiento.actualiza_ruta(rival.current, rival.nodogoal)
            self.primero = False
            return

        if rival.current == self.lista_ruta_rival[0]:
            self.lista_ruta_rival.pop(0)
        else:
            self.lista_ruta_rival = self.pensamiento.actualiza_ruta(rival.current, rival.nodogoal)
        if self.current == rival.nodogoal and self.primero == False:
            self.lista_ruta = [self.current - 1]
            return
        if self.lista_ruta == []:
            self.lista_ruta = self.pensamiento.actualiza_ruta(self.current, self.nodogoal)
            return
        if len(self.lista_ruta) < len(self.lista_ruta_rival):
            return
        if self.alcostado(self.current, rival.current, tablero.n) is True:
            print("llego :v")
            self.lista_ruta = self.pensamiento.actualiza_ruta(self.current, rival.nodogoal)
            return
        if self.alcostado(self.lista_ruta[0], rival.current, tablero.n) is True:
            self.lista_ruta = self.pensamiento.actualiza_ruta(self.current, rival.nodogoal)
            return

In [ ]:
def alcostado(self, nodo1, nodo2, n):
        return nodo1 - 1 is nodo2 or nodo1 + 1 is nodo2 or nodo1 + n is nodo2 or nodo1 - n is nodo2

In [ ]:
def validar_direccion(self, nodoI, nodoF, n):
        validar = nodoI - nodoF
        if (validar == n):
            return 0  # arriba a abajo
        elif (validar == -n):
            return 1  # arriba a abajo
        elif (validar == -1):
            return 2  # izquierda a derecha
        elif (validar == 1):
            return 3  # derecha a izquierda

In [ ]:
def mueve(self, enemigo, tablero):
        inicial = self.current
        self.current = self.lista_ruta[0]
        self.lista_ruta.pop(0)
        if self.current == enemigo.current:
            direccion = self.validar_direccion(inicial, self.current, tablero.n)
            if direccion is 0:
                if self.current - tablero.n >= 0:
                    if tablero.conectados(self.current, self.current - tablero.n):
                        self.current -= tablero.n
                    else:
                        pass
                else:
                    self.current += 1
                    self.lista_ruta = self.pensamiento.actualiza_ruta(self.current, self.nodogoal)
                    return self.current == self.nodogoal
            elif direccion is 1:
                if self.current + tablero.n < tablero.n:
                    if tablero.conectados(self.current, self.current + tablero.n):
                        self.current += tablero.n
                    else:
                        pass
                else:
                    self.current += 1
                    self.lista_ruta = self.pensamiento.actualiza_ruta(self.current, self.nodogoal)
                    return self.current == self.nodogoal
            elif direccion is 2:
                if tablero.conectados(self.current, self.current + 1):
                    self.current += 1
                else:
                    pass
            elif direccion is 3:
                if tablero.conectados(self.current, self.current - 1):
                    self.current -= 1
                else:
                    pass
            if self.current == self.lista_ruta[0]:
                self.lista_ruta.pop(0)
            else:
                self.lista_ruta = self.pensamiento.actualiza_ruta(self.current, self.nodogoal)
        return self.current == self.nodogoal

In [ ]:
def graficar(self, pantalla, pygame, n, lado, color):
        x = self.current % n
        y = self.current // n
        x1 = self.nodogoal % n
        y1 = self.nodogoal // n
        if self.primero == False:
            pygame.draw.rect(pantalla, blue, (x1 * lado, y1 * lado, lado, lado), 0)
        pygame.draw.ellipse(pantalla, color, (x * lado, y * lado, lado, lado), 0)